In [ ]:
import math
import pandas as pd
import numpy as np
import gc
class bm25_Scorer(object):
    def __init__(self, docs):
        self.docs = docs
        self.D = len(self.docs)
        self.avgdl = sum([len(doc) for doc in self.docs]) / self.D
        self.df = {}
        self.idf = {}
        self.k1 = 1.5
        self.b = 0.75
        self.initialize()

    def initialize(self, ngram=1):
        for doc in self.docs:
            word_set = set(doc)
            for word in word_set:
                if word not in self.df:
                    self.df[word] = 1
                self.df[word] += 1
        for k, v in self.df.items():
            self.idf[k] = math.log(self.D - v + 0.5) - math.log(v + 0.5)
    def sim(self, doc1, doc2):
        score = 0
        doc1=doc1.split()
        doc2=doc2.split()
        d = len(doc2)
        loc = self.build_local_df(doc2)
        for word in doc1:
            if word not in loc:
                continue
            score += (self.idf[word] * loc[word] * (self.k1 + 1)
                      / (loc[word] + self.k1 * (1 - self.b + self.b * d
                                                / self.avgdl)))
        return score

    def build_local_df(self, doc):
        tmp = {}
        for word in doc:
            if not word in tmp:
                tmp[word] = 0
            tmp[word] += 1
        return tmp

    def add_ngram(self, n):
        idx = 0
        for doc in self.docs:
            ngram = self.generate_ngram(n, self.titles[idx])
            seg_list = seg_list + ngram
            idx += 1

    def generate_ngram(self, n, sentence):
        return [sentence[i:i + n] for i in range(0, len(sentence) - 1)]

In [ ]:
df_train =pd.read_csv('/home/kesci/input/bytedance/train_final.csv',names=['qId','q','aId','a','target'],nrows=50000000)
df_test =pd.read_csv('/home/kesci/input/bytedance/bytedance_contest.final_2.csv',names=['qId','q','aId','a'])

In [ ]:
df_train.drop(['qId','aId','target'],axis=1,inplace=True)
df_test.drop(['qId','aId'],axis=1,inplace=True)
df_test['a']=df_test['a'].apply(lambda x:x[:-1])
docs=pd.concat([df_train['q'],df_train['a'],df_test['q'],df_test['a']])

In [ ]:
docs=pd.DataFrame(docs)
docs.columns=['doc']
docs=docs.drop_duplicates(["doc"],keep="first")#去重
docs['doc']=docs['doc'].apply(lambda x:x.split())
docs=docs['doc'].tolist() 
bm25Scorer = bm25_Scorer(docs=docs)

In [ ]:
df_train['bm25_q1_to_q2'] = df_train[['q', 'a']].apply(lambda row: bm25Scorer.sim(row['q'], row['a']), axis=1)
df_train['bm25_q2_to_q1'] = df_train[['q', 'a']].apply(lambda row: bm25Scorer.sim(row['a'], row['q']), axis=1)
df_test['bm25_q1_to_q2'] = df_test[['q', 'a']].apply(lambda row: bm25Scorer.sim(row['q'], row['a']), axis=1)
df_test['bm25_q2_to_q1'] = df_test[['q', 'a']].apply(lambda row: bm25Scorer.sim(row['a'], row['q']), axis=1)

In [ ]:
df_train.drop(['q','a'],axis=1,inplace=True)
df_test.drop(['q','a'],axis=1,inplace=True)
train.to_csv('/home/kesci/work/train_feature_bm25_final.csv',index=False)
test.to_csv('/home/kesci/work/test_feature_bm25_final.csv',index=False)